In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score,f1_score

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
cols = ['sentiment','id','date','query_string','user','text']
data = pd.read_csv("data/train.csv",header=None, names=cols, encoding = "ISO-8859-1")

In [3]:
data.head()

,sentiment,id,date,query_string,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
sentiment       1600000 non-null int64
id              1600000 non-null int64
date            1600000 non-null object
query_string    1600000 non-null object
user            1600000 non-null object
text            1600000 non-null object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [5]:
data['query_string'].value_counts()

NO_QUERY    1600000
Name: query_string, dtype: int64

*query_string is a constant column so we can drop that. id, date and user columns does not also help
in finding the tweet sentiment.*

In [6]:
data.drop(['id','date','query_string','user'],axis=1,inplace=True)

In [7]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [8]:
data['sentiment'].value_counts()

4    800000
0    800000
Name: sentiment, dtype: int64

In [9]:
data.loc[data['sentiment']> 0, ['sentiment']] = 1

In [10]:
data['sentiment'].value_counts()

1    800000
0    800000
Name: sentiment, dtype: int64

# Data Preprocessing

In [11]:
stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself'
             , 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself','they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that',
             "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and',
             'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'for','with', 'about', 'between', 'through', 'during', 'before', 'after', 'below',
             'to', 'from', 'up', 'in', 'out', 'on', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all','both', 'each', 'more',
             'most', 'other', 'some', 'such', 'same', 'so', 'than', 'very', 's', 't', 'can', 'will', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y']

data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stopwords)]))

# Data Splitting

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(data.text
, data.sentiment, test_size=.04, random_state=100)
x_validation, x_test, y_validation, y_test =  train_test_split(x_validation_and_test,
y_validation_and_test, test_size=.5, random_state=100)

In [13]:
print(x_train.shape, x_train[y_train>0].shape, x_train[y_train==0].shape)
print(x_validation.shape, x_validation[y_validation>0].shape, x_validation[y_validation==0].shape)
print(x_test.shape, x_test[y_test>0].shape, x_test[y_test==0].shape)

(1536000,) (768081,) (767919,)
(32000,) (15946,) (16054,)
(32000,) (15973,) (16027,)


# Count Vectorization

In [14]:
def eval_metric(prediction, truth):
    print("recall:       {}".format(recall_score(prediction, truth)))
    print("precision:    {}".format(precision_score(prediction, truth)))
    print("f1_score:     {}".format(f1_score(prediction, truth)))
    print("\nconfusion_matrix:\n",confusion_matrix(prediction, truth))

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

CVec = CountVectorizer()
num_of_features = [10000, 20000, 50000]
for n_features in num_of_features:
    
    CVec.set_params(max_features = n_features)
    CVec.fit(x_train)
    
    x_train_vector = CVec.transform(x_train)
    x_validation_vector = CVec.transform(x_validation)
    
    logreg = LogisticRegression()
    logreg.fit(x_train_vector, y_train)
    lr_prediction = logreg.predict(x_validation_vector)
    eval_metric(lr_prediction, y_validation)

recall:       0.7797156851033329
precision:    0.8186379029223629
f1_score:     0.7987028879099364

confusion_matrix:
 [[12366  2892]
 [ 3688 13054]]
recall:       0.7833863513594622
precision:    0.8185124796187132
f1_score:     0.8005642960100592

confusion_matrix:
 [[12445  2894]
 [ 3609 13052]]
recall:       0.7862586186040885
precision:    0.8152514737238179
f1_score:     0.8004926108374385

confusion_matrix:
 [[12520  2946]
 [ 3534 13000]]
